# SQL Database - Connect using Key Vault

In the previous lession ([Key Vault-backed Secret Scopes]($./08-Key-Vault-backed-secret-scopes)), you created a Key Vault-backed secret scope for Azure Databricks, and securely stored your Azure SQL username and password within.

In this lesson, you will use the Azure SQL Database secrets that are securely stored within the Key Vault-backed secret scope to connect to your Azure SQL Database instance, using the JDBC drivers that come with Databricks Runtime version 3.4 and above. Next, you will use the DataFrame API to execute SQL queries and control the parallelism of reads through the JDBC interface.

You will be using a new data set, as the customer has decided to make their e-commerce SQL data available for querying and processing within Azure Databricks. You will prove that you can successfully read data from and write data to the database using the JDBC driver.

## Access Key Vault secrets and configure JDBC connection

In the previous lesson, you created two secrets in Key Vault for your Azure SQL Database instance: **sql-username** and **sql-password**. You should have also made note of the Azure SQL Server host name and Database name. If you do not have this information, take a moment to retrieve those details from the Azure portal.

In [3]:
%scala
val jdbcUsername = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-username")
val jdbcPassword = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-password")

Notice that the values of `jdbcUsername` and `jdbcPassword` when printed out are `[REDACTED]`. This is to prevent your secrets from being exposed.

The next step is to ensure the JDBC driver is available.

In [5]:
%scala
Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")

Now create a connection to the Azure SQL Database using the username and password from Key Vault, as well as the SQL Server host name and database name.

> The **host name** value will be in the following format: `<YOUR_SERVER_NAME>.databrickssqlserver.database.windows.net`.

In [7]:
# Create input widgets to store the host name and database values. This will allow us to access those same values from cells that use different languages.
# Execute this cell to display the widgets on top of the page, then fill the information before continuing to the next cell.
dbutils.widgets.text("hostName", "", "SQL Server Host Name")
dbutils.widgets.text("database", "", "Database Name")

In [8]:
%scala
val jdbcHostname = dbutils.widgets.get("hostName")
val jdbcPort = 1433
val jdbcDatabase = dbutils.widgets.get("database")

// Create the JDBC URL without passing in the user and password parameters.
val jdbcUrl = s"jdbc:sqlserver://${jdbcHostname}:${jdbcPort};database=${jdbcDatabase}"

// Create a Properties() object to hold the parameters.
import java.util.Properties
val connectionProperties = new Properties()

connectionProperties.put("user", s"${jdbcUsername}")
connectionProperties.put("password", s"${jdbcPassword}")

Make sure you can connect to the Azure SQL Database, using the JDBC driver.

In [10]:
%scala
val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
connectionProperties.setProperty("Driver", driverClass)

If there are no errors, then the connection was successful.

## Read data from JDBC

To start, we'll use a single JDBC connection to pull an Azure SQL Database table into the Spark environment. Your queries will be executed through the DataFrame API, providing a consistent experience whether you are querying flat files, Databricks tables, Azure Cosmos DB, SQL Server, or any other of the number of data sources you can access from Databricks.

In [13]:
%scala
val product_table = spark.read.jdbc(jdbcUrl, "SalesLT.Product", connectionProperties)

Spark automatically reads the schema from the database table and maps its types back to Spark SQL types. You can see the schema using the `printSchema` command:

In [15]:
%scala
product_table.printSchema

Now that we have our DataFrame, we can run queries against the JDBC table. Below, we are calculating the average list price by product category.

In [17]:
%scala
display(product_table.select("ProductCategoryID", "ListPrice")
        .groupBy("ProductCategoryID")
        .avg("ListPrice"))

It is possible to save the DataFrame out to a temporary view. This opens opportunities, such as querying using the SQL syntax, as you will see a few cells below.

In [19]:
%scala
product_table.createOrReplaceTempView("Product")
display(spark.sql("SELECT * FROM Product"))

Use the `%sql` magic to change the language of a cell to SQL. In the cell below, execute a SQL query that performs the same average `ListPrice` and product category grouping that was done using the DataFrame earlier.

In [21]:
%sql
select ProductCategoryID, AVG(ListPrice) from Product
Group By ProductCategoryID

You can also use the JDBC driver using Python. The cell below establishes a JDBC connection to the Azure SQL Database just as was done earlier using Scala.

Notice that you must define new parameters for the host name, database, username, and password. This is because those variables were defined using a different language earlier, which are inaccessible by a different language engine.

In [23]:
jdbcHostname = dbutils.widgets.get("hostName")
jdbcDatabase = dbutils.widgets.get("database")
jdbcPort = 1433

# Retrieve the database username and password from Key Vault
jdbcUsername = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-username")
jdbcPassword = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-password")

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

Now that the JDBC connection has been created in Python, let's query the JDBC connections across multiple workers. You do this by setting the `numPartitions` property. This determines how many connections used to push data through the JDBC API.

In [25]:
df = spark.read.jdbc(url=jdbcUrl, table='SalesLT.SalesOrderDetail', properties=connectionProperties, column='SalesOrderDetailID', lowerBound=1, upperBound=100000, numPartitions=100)
display(df)

## Challenge

Write new records to the `SalesLT.ProductModel` Azure SQL Database table.

The first step is to return a count of records in the `SalesLT.ProductModel` table. We've provided the code to do this as a pushdown query (where you push down an entire query to the database and return just the result) below.

In [27]:
productmodel_count_query = "(select COUNT(*) count from SalesLT.ProductModel) count"
productmodel_count = spark.read.jdbc(url=jdbcUrl, table=productmodel_count_query, properties=connectionProperties)
display(productmodel_count)

The second step is to define a schema that matches the data types in SQL. Since the primary key is an automatically generated identity column, you must exclude it from your schema since you will not be setting the value.

**Note:** You can get the schema with data types by retrieving the first row into a new DataFrame and printing the schema:

```python
productmodel_df = spark.read.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", properties=connectionProperties).limit(1)
productmodel_df.printSchema
```

In [29]:
from pyspark.sql.types import *

schema = StructType([
  StructField("Name", StringType(), True),
  StructField("CatalogDescription", StringType(), True),
  StructField("rowguid", StringType(), True),
  StructField("ModifiedDate", TimestampType(), True)
])

Now, write new rows to the table, applying the schema from the `productModel_df` DataFrame.

In [31]:
# Import libraries to generate UUIDs (GUIDs), get the current date/time, set the save mode, and to use the Row object
import uuid
import datetime
from pyspark.sql import Row
newRows = [
  Row("Biking Shorts","They make you ride faster and look awesome doing it!", str(uuid.uuid4()), datetime.datetime.now()),
  Row("Biking Shirt","Glide through the atmosphere like Phoenix rising from the ashes", str(uuid.uuid4()), datetime.datetime.now())
]
parallelizeRows = spark.sparkContext.parallelize(newRows)
new_df = spark.createDataFrame(parallelizeRows, schema)

# TODO

# Add code here to write to the SalesLT.ProductModel table

new_df.write.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", mode="append", properties=connectionProperties)

Do a final count on the `SalesLT.ProductModel` table to make sure the two new rows were inserted. You can also use the Query Editor in the Azure SQL Database portal interface to view your new rows there.

In [33]:
# TODO

# Get a count of rows once more. The value should be 130
productmodel_count = spark.read.jdbc(url=jdbcUrl, table=productmodel_count_query, properties=connectionProperties)
display(productmodel_count)